# Train a CNN with a .fits file as input

In [2]:
# imports
from __future__ import print_function
import keras
from keras import utils as np_utils
import tensorflow
from keras import datasets, layers, models
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


generate_neg_dataset() and generate_pos_dataset()

In [18]:
import joblib
from skimage.transform import resize
from skimage.io import imread
from init_modules import *


# Set the path to the data directory
filenames = glob.glob('data/fits/pos/*.fits')
data_set = {}

im_size = 100
shift_interval = 1
gaussians = []


# This function finds the position of the object in the image
def find_object_pos(file):
    cd = ClustarData(path=file, group_factor=0)
    if len(cd.groups) > 0:
        disk = cd.groups[0]
        bounds = disk.image.bounds
        x = (bounds[2] + bounds[3])/2
        y = (bounds[0] + bounds[1])/2
        print('alwkfnlawnflkn')
        return (x, y)
    else:
        print("No object found in {}".format(file))
        return None


# This function crops the image to the size of the object
def init_cropped_images():
    for file in filenames:
        img_data = fits.getdata(file)
        object_pos = find_object_pos(file)

        if object_pos != None:
            # Data shape is (1, 1, x, y) we want it to be (x, y)
            img_data.shape = (img_data.shape[2], img_data.shape[3])

            # Set the size of the crop in pixels
            crop_size = units.Quantity((im_size, im_size), units.pixel)

            img_crop = Cutout2D(img_data, object_pos, crop_size)

            gaussians.append(img_crop)


# This function rotates the image by a random angle
def rotate_disk(disk_to_rotate, angle):

    # Rotate the disk
    rotated_disk = rotate(disk_to_rotate, angle)
    # Since rotating pads the image, we need to crop it to the original size
    x, y = (len(rotated_disk[0]), len(rotated_disk))

    shift_interval = 8
    si = shift_interval + 1

    rand_x_shift = random.randint(-shift_interval, shift_interval)
    rand_y_shift = random.randint(-shift_interval, shift_interval)

    (x_lower, x_upper) = int((x/2 - im_size/2)) + \
        rand_x_shift, int(x/2 + im_size/2) + rand_x_shift
    (y_lower, y_upper) = int((y/2 - im_size/2)) + \
        rand_y_shift, int(y/2 + im_size/2) + rand_y_shift

    return rotated_disk[(x_lower+si):(x_upper-si), (y_lower+si):(y_upper-si)]


# This function flips the image horizontally, vertically or both
def flip_disk(disk_to_flip):

    flipped_disk = disk_to_flip

    if bool(random.getrandbits(1)):
        flipped_disk = np.fliplr(flipped_disk)

    if bool(random.getrandbits(1)):
        flipped_disk = np.flipud(flipped_disk)

    if bool(random.getrandbits(1)):
        flipped_disk = np.flip(flipped_disk)

    return flipped_disk


# This function augments the image by rotating and flipping it
def augment_disk(disk):
    angle = random.randint(0, 360)
    return rotate_disk(flip_disk(disk), angle)


# This function generates the positive dataset
def generate_pos_dataset(augmentations_per_gaussian):
    count = 0
    pos_dataset = []
    for gaussian in gaussians:
        for i in range(0, augmentations_per_gaussian):
            zscale = ZScaleInterval(contrast=0.25, nsamples=1)
            # Augment the data and add it to the dataset as png
            plt.figure()
            plt.imshow(zscale(augment_disk(gaussian.data)),
                       origin='lower', cmap='rainbow')
            # plt.colorbar()
            # plt.title("Augmented gaussian")
            # plt.show()

            pos_dataset.append(zscale(augment_disk(gaussian.data)))
    return pos_dataset


# This function generates the negative dataset

def generate_neg_dataset(augmentations_per_gaussian):
    neg_dataset = []
    im_center = im_size/2
    y, x = np.mgrid[0:im_size, 0:im_size]
    for i in range(0, len(filenames)):
        for j in range(0, augmentations_per_gaussian):
            rand_x_shift = random.randint(-shift_interval, shift_interval)
            rand_y_shift = random.randint(-shift_interval, shift_interval)
            data = Gaussian2D(1, im_center + rand_x_shift,
                              im_center +
                              rand_y_shift, random.randrange(5, 20),
                              random.randrange(5, 20), theta=random.randrange(0, 2))(x, y)

            zscale = ZScaleInterval(contrast=0.25, nsamples=1)
            neg_dataset.append(zscale(augment_disk(data)))
    return neg_dataset


In [17]:
# Hyper-parameters data-loading and formatting
batch_size = 128
num_classes = 2
epochs = 3

img_rows, img_cols = im_size, im_size

x_train = np.array(generate_neg_dataset(1) + generate_pos_dataset(1))
lbl_train = [0] * len(generate_neg_dataset(1)) + [1] * len(generate_pos_dataset(1))

print(len(x_train))

x_test = np.array(generate_neg_dataset(1) + generate_pos_dataset(1))
lbl_test = [0] * len(generate_neg_dataset(1)) + [1] * len(generate_pos_dataset(1))

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


8


ValueError: cannot reshape array of size 53792 into shape (8,75,75,1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = keras.utils.np_utils.to_categorical(lbl_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(lbl_test, num_classes)
